# Thiết kế cơ sở dữ liệu cho hệ thống hồ sơ bệnh án điện tử (EMR)


- ER Diagram for EMR database
![Alt text](EMR.jpg)

In [3]:
import os
from dotenv import load_dotenv
import pymysql

load_dotenv(".credential")

conn = pymysql.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    database="emrdb"
)

cursor = conn.cursor()

In [4]:
# -- Bảng Patient (Bệnh nhân)
cursor.execute("""
CREATE TABLE IF NOT EXISTS PATIENT (
    PatientID INT PRIMARY KEY,
    FullName VARCHAR(100) NOT NULL,
    BirthDate DATE NOT NULL,
    Gender VARCHAR(10) NOT NULL,
    Address TEXT,
    MedicalID VARCHAR(50) UNIQUE NOT NULL
);
""")

# -- Bảng Visit (Lượt khám)
cursor.execute("""
CREATE TABLE IF NOT EXISTS VISIT (
    VisitID INT PRIMARY KEY,
    PatientID INT NOT NULL,
    VisitDate DATE NOT NULL,
    Symptoms TEXT,
    Diagnosis TEXT,
    ImagingResults TEXT,
    FOREIGN KEY (PatientID) REFERENCES PATIENT(PatientID) ON DELETE CASCADE
);
""")

# -- Bảng Prescription (Đơn thuốc)
cursor.execute("""
CREATE TABLE IF NOT EXISTS PRESCRIPTION (
    PrescriptionID INT PRIMARY KEY,
    VisitID INT NOT NULL,
    MedicationName VARCHAR(100) NOT NULL,
    Dosage VARCHAR(50),
    Frequency VARCHAR(50),
    TreatmentPlan TEXT,
    Procedures TEXT,
    FollowUp TEXT,
    FOREIGN KEY (VisitID) REFERENCES VISIT(VisitID) ON DELETE CASCADE
);
""")

# -- Bảng Test (Xét nghiệm)
cursor.execute("""
CREATE TABLE IF NOT EXISTS TEST (
    TestID INT PRIMARY KEY,
    VisitID INT NOT NULL,
    TestType VARCHAR(100),
    Result TEXT,
    TestDate DATE,
    ResponsibleDoctor VARCHAR(100),
    ReceivingTime TIMESTAMP,
    MedicalFacility VARCHAR(100),
    Department VARCHAR(100),
    FOREIGN KEY (VisitID) REFERENCES VISIT(VisitID) ON DELETE CASCADE
);
""")

# Thêm dữ liệu vào bảng
# Insert Patients
cursor.execute("""
INSERT INTO PATIENT (PatientID, FullName, BirthDate, Gender, Address, MedicalID)
VALUES
(1, 'Nguyễn Văn A', '1990-01-15', 'Nam', 'Hà Nội', 'MED10001'),
(2, 'Trần Thị B', '1985-07-23', 'Nữ', 'TP. Hồ Chí Minh', 'MED10002'),
(3, 'Lê Văn C', '1978-03-02', 'Nam', 'Đà Nẵng', 'MED10003'),
(4, 'Phạm Thị D', '1992-09-12', 'Nữ', 'Hải Phòng', 'MED10004'),
(5, 'Đỗ Văn E', '2000-12-25', 'Nam', 'Cần Thơ', 'MED10005');
""")

# Insert Visits
cursor.execute("""
INSERT INTO VISIT (VisitID, PatientID, VisitDate, Symptoms, Diagnosis, ImagingResults)
VALUES
(101, 1, '2024-05-10', 'Sốt, ho', 'Cảm cúm', 'X-quang phổi bình thường'),
(102, 2, '2024-05-11', 'Đau bụng', 'Viêm dạ dày', 'Siêu âm bụng bình thường'),
(103, 3, '2024-05-12', 'Đau lưng', 'Căng cơ', 'X-quang cột sống bị lệch nhẹ'),
(104, 4, '2024-05-13', 'Chóng mặt', 'Huyết áp thấp', 'Chụp CT não bình thường'),
(105, 5, '2024-05-14', 'Phát ban', 'Viêm da', 'Sinh thiết da âm tính');
""")

# Insert Prescriptions
cursor.execute("""
INSERT INTO PRESCRIPTION (PrescriptionID, VisitID, MedicationName, Dosage, Frequency, TreatmentPlan, Procedures, FollowUp)
VALUES
(201, 101, 'Paracetamol', '500mg', 'Ngày 2 lần', 'Nghỉ ngơi và uống nhiều nước', 'Không có', 'Tái khám sau 3 ngày'),
(202, 102, 'Omeprazole', '20mg', 'Ngày 1 lần', 'Ăn uống hợp lý', 'Nội soi dạ dày', 'Tái khám sau 1 tuần'),
(203, 103, 'Ibuprofen', '400mg', 'Ngày 3 lần', 'Vật lý trị liệu', 'Xoa bóp', 'Tái khám sau 1 tuần'),
(204, 104, 'Viên muối', '1 viên', 'Ngày 2 lần', 'Bổ sung nước', 'Xét nghiệm máu', 'Tái khám sau 5 ngày'),
(205, 105, 'Kem Hydrocortisone', 'Bôi một lớp mỏng', 'Ngày 2 lần', 'Tránh dị ứng', 'Cạo da kiểm tra', 'Tái khám sau 1 tuần');
""")

# Insert Tests
cursor.execute("""
INSERT INTO TEST (TestID, VisitID, TestType, Result, TestDate, ResponsibleDoctor, ReceivingTime, MedicalFacility, Department)
VALUES
(301, 101, 'X-quang', 'Không phát hiện bất thường', '2024-05-10', 'BS. Nguyễn', '2024-05-10 09:00:00', 'Bệnh viện Bạch Mai', 'Chẩn đoán hình ảnh'),
(302, 102, 'Siêu âm bụng', 'Bình thường', '2024-05-11', 'BS. Trần', '2024-05-11 10:30:00', 'Bệnh viện Chợ Rẫy', 'Tiêu hóa'),
(303, 103, 'X-quang cột sống', 'Lệch nhẹ', '2024-05-12', 'BS. Lê', '2024-05-12 11:00:00', 'Bệnh viện Đà Nẵng', 'Cơ xương khớp'),
(304, 104, 'CT não', 'Không có vấn đề', '2024-05-13', 'BS. Phạm', '2024-05-13 13:00:00', 'Bệnh viện Hải Phòng', 'Thần kinh'),
(305, 105, 'Sinh thiết da', 'Âm tính', '2024-05-14', 'BS. Đỗ', '2024-05-14 14:30:00', 'Bệnh viện Cần Thơ', 'Da liễu');
""")

conn.commit()

In [5]:
# Close Connection
cursor.close()
conn.close()